In [1]:
import torch
import numpy as np
from utils import get_dataset, get_net, get_strategy
import pickle

In [2]:
samples = np.load("samples_bert.npy")

In [3]:
samples

array([  0, 100, 161, 221, 282, 344, 409, 474])

In [4]:
# fix random seed
# np.random.seed(1)
# torch.manual_seed(1)
torch.backends.cudnn.enabled = False
name = "BERT"
strat = "LeastConfidence"
# device
acc_list_kc_list = []
agr_list_kc_list = []
kl_list_kc_list = []

for i in range(10):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # dataset = Data(unlab_x, unlab_y, test_x, test_y, cifar_handler) 
    dataset = get_dataset(name)    # load dataset
    net = get_net(name, device, y_num = 10)            # load network
    strategy = get_strategy(strat)(dataset, net)  # load strategy
    initial = samples[1]
    # start experiment
    dataset.initialize_labels(initial)
    print(f"number of labeled pool: {initial}")
    print(f"number of unlabeled pool: {dataset.n_pool-initial}")
    print(f"number of testing pool: {dataset.n_test}")
    print()

    # round 0 accuracy
    print("Round 0")
    strategy.train()
    preds, agr, kl = strategy.predict(dataset.get_test_data())
    acc_list_kc = []
    agr_list_kc = []
    kl_list_kc = []
    acc = dataset.cal_test_acc(preds)*100
    acc_list_kc.append(acc)
    agr_list_kc.append(agr)
    kl_list_kc.append(kl)
    print(f"Round 0 testing accuracy: {acc} %")
    print("Samples:", samples[1])
    for rd in range(1, len(samples)-1):
        print(f"Round {rd}")
        budget = samples[rd+1] - samples[rd]
        # query
        query_idxs = strategy.query(budget)
        # update labels
        strategy.update(query_idxs)
        strategy.train()

        # calculate accuracy
        preds, agr, kl = strategy.predict(dataset.get_test_data())
        acc = dataset.cal_test_acc(preds)*100
        print('\n')
        print(f"Round {rd} testing accuracy: {acc} %")
        acc_list_kc.append(acc)
        agr_list_kc.append(agr)
        kl_list_kc.append(kl)
        print("Samples:", samples[rd+1])
        torch.save(strategy.net.clf.state_dict(), "./extracted_models/"+name+"_"+strat+"_"+str(i)+str(3)+".pt")
    acc_list_kc_list.append(acc_list_kc)
    agr_list_kc_list.append(agr_list_kc)
    kl_list_kc_list.append(kl_list_kc)

    with open("./results/acc"+name+strat+str(3), "wb") as fp:
      pickle.dump(acc_list_kc_list, fp)
    with open("./results/agr"+name+strat+str(3), "wb") as fp:
      pickle.dump(agr_list_kc_list, fp)
    with open("./results/kl"+name+strat+str(3), "wb") as fp:
      pickle.dump(kl_list_kc_list, fp)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


number of labeled pool: 100
number of unlabeled pool: 7000
number of testing pool: 290

Round 0


  0%|                                                                         | 0/3 [00:00<?, ?it/s]

Train accuracy =  tensor(0.2167, device='cuda:0')
saving model


 33%|█████████████████████▋                                           | 1/3 [00:13<00:27, 13.90s/it]

...

Train accuracy =  tensor(0.8842, device='cuda:0')
saving model


 67%|███████████████████████████████████████████▎                     | 2/3 [00:59<00:28, 28.56s/it]

Train accuracy =  tensor(0.8947, device='cuda:0')
saving model


100%|█████████████████████████████████████████████████████████████████| 3/3 [01:35<00:00, 31.79s/it]
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 37/37 [00:07<00:00,  5.07it/s]




Round 6 testing accuracy: 61.03448275862069 %
Samples: 474


In [17]:
acc_list_kc_list

[[19.310344827586206,
  31.724137931034484,
  46.55172413793103,
  46.55172413793103,
  55.51724137931034,
  51.37931034482759,
  60.0],
 [24.137931034482758,
  42.06896551724138,
  37.58620689655172,
  36.55172413793103,
  66.20689655172414,
  86.89655172413792,
  84.82758620689656],
 [21.03448275862069,
  37.93103448275862,
  38.96551724137931,
  43.44827586206896,
  45.17241379310345,
  61.37931034482759,
  65.51724137931035],
 [19.96164712105796,
  29.50883438950384,
  48.512695143976,
  44.679072525800564,
  54.39057618246424,
  49.10938790889123,
  60.325002454240675],
 [22.540053983224638,
  41.111452864068816,
  39.731080579076625,
  34.56282044048536,
  65.30283013005935,
  86.31888070768126,
  87.13711713103736],
 [21.379310344827587,
  22.758620689655174,
  27.241379310344826,
  36.206896551724135,
  59.310344827586206,
  62.758620689655174,
  84.48275862068965],
 [19.310344827586206,
  19.310344827586206,
  37.58620689655172,
  37.93103448275862,
  46.89655172413793,
  52.7